# Bond Calculator

This is an example of a bond calculator powered by IPA.

In [1]:
import refinitiv.data as rd
import refinitiv.data.content.ipa.financial_contracts as rdf
from refinitiv.data.content.ipa.financial_contracts import bond
import ipywidgets as widgets
import pandas as pd
from datetime import datetime
import dateutil.parser

rd.open_session()

<refinitiv.data.session.Definition object at 0x14c07ab20 {name='workspace'}>

In [2]:
# CALCULATIONS & FUNCTIONS
ignore_events = False
redemptionDateType = None
redemption_type_map = {'Maturity': 'RedemptionAtMaturityDate',
                       'Call': 'RedemptionAtCallDate',
                       'Put': 'RedemptionAtPutDate',
                       'Best': 'RedemptionAtBestDate',
                       'Worst': 'RedemptionAtWorstDate'}

changed_parameters = []
analytics_df = None


def parse_date(tz):
    d = dateutil.parser.parse(tz)
    return d.date()


def bond_calculate(universe, parameters):
    global instrument_description_text, trade_date, settlement_date, settle_wd, clean_price_text, clean_price_text
    global price_type_dropdown, yield_input, yield_type_dropdown, redemption_date, redemption_price, accrued_out, dirty_price_text, redemptionDateType
    global coupon_dates_text, coupon_frequency_text, ex_dividend_text, coupon_rate_text
    print(universe)
    df = rdf.Definitions(
        universe=[universe],
        pricing_parameters=parameters
    ).get_data().data.df
    
    instrument_description_text.value = df['InstrumentDescription'][0]
    trade_date.value = df['MarketDataDate'][0]
    settlement_date.value = df['ValuationDate'][0]
    settle_wd.value = int(df['SettlementConvention'][0][0])
    clean_price_text.value = df['CleanPricePercent'][0]
    if df['PriceSide'][0] == None:
        price_side = 'Custom'
    else:
        price_side = df['PriceSide'][0]
    price_type_dropdown.value = price_side
    yield_input.value = df['YieldPercent'][0]
    yield_type_dropdown.value = df['RedemptionYieldType'][0].replace(
        'YieldTo', '')
    redemption_date.value = df['RedemptionDate'][0].strftime('%Y-%m-%d')
    redemptionDateType = df['RedemptionDateType'][0]
    redemption_price.value = str(df['RedemptionPrice'][0])
    accrued_out.value = str(df['Accrued'][0])
    dirty_price_text.value = df['DirtyPricePercent'][0]
    coupon_dates_text.value = str(
        df['PreviousCouponDate'][0])+'-'+str(df['NextCouponDate'][0])
    coupon_frequency_text.value = df['InterestPaymentFrequency'][0]
    ex_dividend_text.value = str(df['NextExDividendDate'][0])
    coupon_rate_text.value = str(df['NextCouponRatePercent'][0])
    analytics_df = df
    return df


def fetch_data(ric):
    global instrument_description_text, trade_date, settlement_date, settle_wd, clean_price_text, clean_price_text
    global price_type_dropdown, yield_input, yield_type_dropdown, redemption_date, redemption_date_type, redemption_price, accrued_out, dirty_price_text
    global coupon_dates_text, coupon_frequency_text, ex_dividend_text, coupon_rate_text
    global analytics_df

    df = bond.Definition(instrument_code=ric).get_data().data.df

    instrument_description_text.value = df['InstrumentDescription'][0]
    trade_date.value = df['MarketDataDate'][0]
    settlement_date.value = df['ValuationDate'][0]
    settle_wd.value = int(df['SettlementConvention'][0][0])
    clean_price_text.value = df['CleanPricePercent'][0]
    if df['PriceSide'][0] == None:
        price_side = 'Custom'
    else:
        price_side = df['PriceSide'][0]
    price_type_dropdown.value = price_side
    yield_input.value = df['YieldPercent'][0]
    yield_type_dropdown.value = df['RedemptionYieldType'][0].replace(
        'YieldTo', '')
    redemption_date.value = str(df['RedemptionDate'][0])
    redemptionDateType = df['RedemptionDateType'][0]
    redemption_price.value = str(df['RedemptionPrice'][0])
    accrued_out.value = str(df['Accrued'][0])
    dirty_price_text.value = df['DirtyPricePercent'][0]
    coupon_dates_text.value = str(
        df['PreviousCouponDate'][0])+'-'+str(df['NextCouponDate'][0])
    coupon_frequency_text.value = df['InterestPaymentFrequency'][0]
    ex_dividend_text.value = str(df['NextExDividendDate'][0])
    coupon_rate_text.value = str(df['NextCouponRatePercent'][0])
    analytics_df = df
    return df


def start():
    global ignore_events
    ignore_events = True
    fetch_data(str(instrument_input.value))
    ignore_events = False
    return


def create_calculation_params(parameters):

    if 'marketDataDate' in changed_parameters:
        market_data_date = str(trade_date.value)
    else:
        market_data_date = None

    if 'valuationDate' in changed_parameters:
        valuation_date = str(settlement_date.value)
    else:
        valuation_date = None

    if 'settlementConvention' in changed_parameters:
        settlement_convention = str(settle_wd.value)+'WD'
        universe = bond.Definition(
            instrument_code = instrument_input.value,
            pricing_parameters = bond.PricingParameters(
                settlement_convention=settlement_convention))
    else:
        settlement_convention = None
        universe = instrument_input.value

    if 'CleanPrice' in changed_parameters:
        clean_price = clean_price_text.value
    else:
        clean_price = None

    if 'priceSide' in changed_parameters:
        if price_type_dropdown.value == 'Custom':
            price_side = None
        else:
            price_side = bond.PriceSide(
                price_type_dropdown.value)
    else:
        price_side = None

    if 'YieldPercent' in changed_parameters:
        yield_percent = yield_input.value
    else:
        yield_percent = None

    if 'RedemptionDateType' in changed_parameters:
        redemption_date_type = bond.RedemptionDateType(
            redemptionDateType)
    else:
        redemption_date_type = None

    if 'DirtyPrice' in changed_parameters:
        dirty_price = dirty_price_text.value
    else:
        dirty_price = None

    calculation_params = bond.PricingParameters(
        market_data_date=market_data_date,
        valuation_date=valuation_date,
        clean_price=clean_price,
        price_side=price_side,
        yield_percent=yield_percent,
        redemption_date_type=redemption_date_type,
        dirty_price=dirty_price
    )

    return universe, calculation_params


def instrument_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        ric = value['new']
        changed_parameters = []
        fetch_data(ric)
        ignore_events = False
    return


def reset(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        ric = instrument_input.value
        changed_parameters = []
        fetch_data(ric)
        ignore_events = False
    return


def trade_date_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        if 'marketDataDate' not in changed_parameters:
            changed_parameters.append('marketDataDate')
            if ('valuationDate' in changed_parameters) & ('settlementConvention' in changed_parameters):
                changed_parameters.remove('settlementConvention')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False
    return


def settle_date_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        if 'valuationDate' not in changed_parameters:
            changed_parameters.append('valuationDate')
            if ('marketDataDate' in changed_parameters) & ('settlementConvention' in changed_parameters):
                changed_parameters.remove('settlementConvention')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False
    return


def settle_wd_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        if not (('valuationDate' in changed_parameters) & ('marketDataDate' in changed_parameters)):
            changed_parameters.append('settlementConvention')
            universe, calculation_params = create_calculation_params(
                changed_parameters)
            bond_calculate(universe, calculation_params)
        else:
            settle_wd.value = value['old']
        ignore_events = False
    return


def clean_price_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        if 'CleanPrice' not in changed_parameters:
            changed_parameters.append('CleanPrice')
            if 'DirtyPrice' in changed_parameters:
                changed_parameters.remove('DirtyPrice')
            if 'YieldPercent' in changed_parameters:
                changed_parameters.remove('YieldPercent')
            if 'priceSide' in changed_parameters:
                changed_parameters.remove('priceSide')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False
    return


def price_type_change(value):
    global ignore_events, changed_parameters
    price_type = value['new']
    if not ignore_events:
        ignore_events = True
        if price_type != 'Custom':
            if 'priceSide' not in changed_parameters:
                changed_parameters.append('priceSide')
            if 'DirtyPrice' in changed_parameters:
                changed_parameters.remove('DirtyPrice')
            if 'YieldPercent' in changed_parameters:
                changed_parameters.remove('YieldPercent')
            if 'CleanPrice' in changed_parameters:
                changed_parameters.remove('CleanPrice')
            universe, calculation_params = create_calculation_params(
                changed_parameters)
            bond_calculate(universe, calculation_params)
        ignore_events = False
    return


def yield_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        if 'YieldPercent' not in changed_parameters:
            changed_parameters.append('YieldPercent')
            if 'DirtyPrice' in changed_parameters:
                changed_parameters.remove('DirtyPrice')
            if 'CleanPrice' in changed_parameters:
                changed_parameters.remove('CleanPrice')
            if 'priceSide' in changed_parameters:
                changed_parameters.remove('priceSide')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False
    return


def yield_type_change(value):
    global ignore_events, changed_parameters
    if not ignore_events:
        ignore_events = True
        redemptionDateType = redemption_type_map[value['new']]
        if 'RedemptionDateType' not in changed_parameters:
            changed_parameters.append('RedemptionDateType')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False

    return


def dirty_price_change(value):
    global ignore_events
    if not ignore_events:
        ignore_events = True
        if 'DirtyPrice' not in changed_parameters:
            changed_parameters.append('DirtyPrice')
            if 'CleanPrice' in changed_parameters:
                changed_parameters.remove('CleanPrice')
            if 'YieldPercent' in changed_parameters:
                changed_parameters.remove('YieldPercent')
            if 'priceSide' in changed_parameters:
                changed_parameters.remove('priceSide')
        universe, calculation_params = create_calculation_params(
            changed_parameters)
        bond_calculate(universe, calculation_params)
        ignore_events = False
    return

In [4]:
# USER INTERFACE
# widgets.Layout(border = 'solid 0.7px blue'))

left_labels_width = '120px'
style = {'description_width': left_labels_width}
layout = {'width': '450px'}
box_layout = {'margin': '0'}

# instrument input
instrument_input = widgets.Text(description='RIC', value='US5YT=RR', continuous_update=False, layout=widgets.Layout(
    margin='0 0 0 0', width='320px'), style={'description_width': '25px'})
instrument_input.observe(instrument_change, names='value')
button_reset = widgets.Button(icon='undo', tooltip='Reset changes',
                              layout=widgets.Layout(padding='0 0 0 0', width='26px'))
button_reset.on_click(reset)
instrument_description_text = widgets.Label('No instrument')

instrument_panel = widgets.HBox([instrument_input, button_reset])
header_box = widgets.VBox([instrument_panel, instrument_description_text],
                          layout=widgets.Layout(margin='0 0 0 0', width='450px'))

# settlement
trade_date = widgets.DatePicker(
    description='Trade date', continuous_update=False, style=style)
trade_date.observe(trade_date_change, names='value')
settlement_date = widgets.DatePicker(
    description='Settlement date', continuous_update=False, style=style)
settlement_date.observe(settle_date_change, names='value')
settle_wd = widgets.IntText(continuous_update=False, layout={'width': '50px'})
settle_wd.observe(settle_wd_change, names='value')
label_wd = widgets.Label('D')
trade_box = widgets.HBox([trade_date])

settlement_rules = widgets.HBox([settlement_date, settle_wd, label_wd])
settlement_box = widgets.VBox([trade_box, settlement_rules])

# pricing
clean_price_text = widgets.FloatText(
    description='Clean price (%)', continuous_update=False, style=style)
clean_price_text.observe(clean_price_change, names='value')
price_type_dropdown = widgets.Dropdown(options=['Bid', 'Ask', 'Mid', 'Custom'],
                                       value='Bid',
                                       continuous_update=False,
                                       layout={'width': '80px'}
                                       )
price_type_dropdown.observe(price_type_change, names='value')

price_box = widgets.HBox([clean_price_text, price_type_dropdown])

yield_input = widgets.FloatText(
    description='Yield (%)', continuous_update=False, style=style)
yield_input.observe(yield_change, names='value')
yield_type_dropdown = widgets.Dropdown(options=['Maturity', 'Put', 'Call', 'Worst', 'Best'],
                                       value='Maturity',
                                       continuous_update=False,
                                       layout={'width': '80px'}
                                       )
yield_type_dropdown.observe(yield_type_change, names='value')

yields_box = widgets.HBox([yield_input, yield_type_dropdown])

redemption_date = widgets.Text(
    description='Redemption date', continuous_update=False, style=style, disabled=True)
redemption_price = widgets.Text(
    description='Redemption price', continuous_update=False, style=style, disabled=True)
redemption_box = widgets.VBox([redemption_date, redemption_price])

accrued_out = widgets.Text(description='Accrued',
                           continuous_update=False, style=style, disabled=True)

dirty_price_text = widgets.FloatText(
    description='Dirty price (%)', continuous_update=False, style=style)
dirty_price_text.observe(dirty_price_change, names='value')

pricing_box = widgets.VBox(
    [price_box, yields_box, redemption_box, accrued_out, dirty_price_text])

# coupon
coupon_dates_text = widgets.Text(
    description='Previous / next', continuous_update=False, style=style, disabled=True)
coupon_frequency_text = widgets.Text(
    description='Frequency', continuous_update=False, style=style, disabled=True)
ex_dividend_text = widgets.Text(
    description='Next ex-dividend', continuous_update=False, style=style, disabled=True)
coupon_rate_text = widgets.Text(
    description='Rate (%)', continuous_update=False, style=style, disabled=True)

coupon_box = widgets.VBox(
    [coupon_dates_text, coupon_frequency_text, ex_dividend_text, coupon_rate_text])

# side panel
accordion_settlement = widgets.Accordion(
    children=[settlement_box], layout=widgets.Layout(margin='0 0 0 0', width='450px'))
accordion_pricing = widgets.Accordion(
    children=[pricing_box], layout=widgets.Layout(margin='0 0 0 0', width='450px'))
accordion_coupon = widgets.Accordion(
    children=[coupon_box], layout=widgets.Layout(margin='0 0 0 0', width='450px'))
accordion_settlement.set_title(0, 'SETTLEMENT')
accordion_pricing.set_title(0, 'PRICING')
accordion_coupon.set_title(0, 'COUPON')
side_panel = widgets.VBox(
    [header_box, accordion_settlement, accordion_pricing, accordion_coupon])
start()
side_panel

In [5]:
rd.close_session()